# Exercises Week 10-11: Graded exercise session (part A)

**Course**: [Life Sciences engineering: genome to function](https://go.epfl.ch/BIO-411) (BIO-411)

**Professors**:  _Gönczy Pierre_, _Naef Felix_, _McCabe Brian Donal_

SSV, MA, 2025-2026


In [ ]:
#import librairies
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.special import gammaln

np.random.seed(42)  # For reproducibility

**Provide answers in the form of code, figures (if relevant) and short descriptions (in markdown cells) in those notebooks. Submit your notebook to Moodle, please make sure to execute every cell before you submit.**

## Exercise 1: Transcriptional bursting

Let's consider the transcriptional process in which an mRNA $x$ is produced at a rate $s$ and degraded at a rate $k$. We will consider a two-state model for the promoter (Random Telegraph process). This means that the promoter can switch from an active ($g=1$) to inactive state ($g=0$) and vice-versa with respective rates $k_{off}$ and $k_{on}$.


The Telegraph process has the following elementary reactions:  
\begin{array}{lll}
(1, n) \longrightarrow (1, n+1) \textrm{ with rate } s \\
(g, n) \longrightarrow (g, n-1) \textrm{ with rate } k\,n \\
(1, n) \longrightarrow (0, n) \textrm{ with rate } k_{off} \\
(0, n) \longrightarrow (1, n) \textrm{ with rate } k_{on}
\end{array}

We describe the states $(g, n)$ with $g\in \{1, 0\}$, $n \in \mathbb{N}$.

*Hint*: In all of the exercises below, use rates that are plausible biologically, especially for $s$ and $k$ (see course). Note that the units here are in minutes.

#### Question 1
1. Modify/complete the code template provided for the Gillespie algorithm to simulate the Telegraph model.

2. Two genes differ by the percentage of time the promoter is ON (gene1: 100%, gene2 : 10%), but the other parameters are the same ($s$, $k$). Let’s modify gene2 so that it expresses at the same level than gene 1, by either increasing its synthesis rate, or decreasing its degradation rate. Let’s say that you also aim to minimize the variance, which scenario is better? For each of the 4 cases (gene1, gene2, gene2 with synthesis rate up, gene2 with degradation rate down), answer with a simulation and show the distribution of the number of molecules and explain your findings.  
*Note*: For each case, report the variance, the mean level of expression, the expected number of molecules, the coefficient of variation of the number of mRNA copies ($CV=\frac{\sigma}{\mu}$) and the probability of the promoter to be in the $ON$ state.  

#### Gillespie template

In [ ]:
import numpy as np

def run_gillespie_telegraph(n0, g0, parameters, t_end, DT):
    """
    Simulates the Random telepgrah process using the Gillespie algorithm.
    Parameters:
    n0 : Initial count of species n.
    g0 : Initial promoter state g (0 or 1).
    parameters: Tuple containing transcription rate s, degradation rate k, switching rates k_on and k_off
    t_end : Total simulation time.
    DT: time interval for returning/printing the output

    Returns:
    n_print : Trajectory of species n counts over time.
    g_print : Trajectory of promoter state g over time.
    t_print : Time points for each recorded reaction.
    """
    # Initialize state variables and time
    n = n0   
    g = g0   
    t = 0    
    tp = 0  

    # Initialize output lists
    n_print=[]  
    g_print=[]  
    t_print=[]  

    # Unpack parameters
    s, k, k_on, k_off = parameters 

    while t < t_end: 
        
        # Define the stoichiometric matrix S (also called state-change matrix) and rate vector for each promoter state
        if g == 1: # Promoter is active 
            S = np.array([
                [-1,  0],  # gene setting to inactive, rate k_off
                [ 0, +1],  # mRNA production, rate s
                [ 0, -1]   # natural mRNA degradation, rate k*n
            ])
            rates = np.array([k_off, s, k*n])
        else: # Promoter is inactive
            S = np.array([
                [+1,  0],  # gene switching to active, rate k_on
                [ 0, -1]   # natural mRNA degradation, rate k*n
            ])
            rates = np.array([k_on, k*n])

        # Calculate total rate and sample time to next reaction
        rates_tot = sum(rates)
        dt = np.random.exponential(scale=1.0 / rates_tot)
        t += dt

        # Determine which reaction occurs next
        rand = np.random.random()        
        rates_cumsum = np.cumsum(rates)/rates_tot

        i=0
        while rand >= rates_cumsum[i]:
            i=i+1

        # Record output at specified DT intervals (equally spaced output)
        while tp < t and t <= t_end:
            n_print.append(n)
            g_print.append(g)
            t_print.append(tp)
            tp += DT
        
        # Update species n counts and promoter state g
        chosen_react=i
        g += S[chosen_react,0]
        n += S[chosen_react,1]

    return n_print, g_print, t_print

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_gillespie_telegraph(n, g, t):
    """
    Plots the Random telepgrah process using the Gillespie algorithm.

    Parameters:
    n: Species n counts over time.
    g: Promoter state g over time.
    t: Time points for each recorded reaction.

    """

    fig, axs = plt.subplots(2, figsize=(15, 5), sharex=True)

    # Plotting the number of molecules n over time
    axs[0].plot(t, n, label = 'n(t)', color='darkblue')
    axs[0].set_ylabel('# molecules n')

    # Plotting the promoter activity over time
    axs[1].vlines(x=t, ymin=np.min(g), ymax=np.array(g), colors='teal', lw=2, label='promoter activity')
    axs[1].set_ylim(np.min(g), np.max(g)+0.1)  # Set y-limits to make the promoter states clear (0 and 1)
    axs[1].set_ylabel('Promoter State')
    
    # Common x-label
    plt.xlabel('time [min]')
    
    plt.show()

In [ ]:
#Plot distribution of number of mRNA molecules
def plot_distribution(n, title=''):
    plt.figure(figsize=(8,5))
    plt.hist(n, bins=50, color='skyblue', edgecolor='black')
    plt.title(title)
    plt.xlabel('Number of mRNA molecules')
    plt.ylabel('Frequency')
    plt.show()

#Helper functions for analysis. For each case, report the variance, the mean level of expression, the expected number of molecules,
#the coefficient of variation of the number of mRNA copies, and the probability of the promoter to be in the ON state.
def compute_stats(n, g):
    mean_n = np.mean(n)
    expected_n = mean_n #Expected number of molecules = mean level of expression
    var_n = np.var(n)
    cv_n = np.sqrt(var_n)/mean_n if mean_n != 0 else 0
    p_on = np.mean([1 if gg==1 else 0 for gg in g])
    return mean_n, expected_n, var_n, cv_n, p_on

#### Gene 1

In [ ]:
#initial conditions for the state (g,n) at time t=0
g_0 = 1 # Initial promoter state
DT = 0.1 # min
#if needed adjust DT so that it makes sense given the timescales of the system (depending on your rates)

#Parameters, these are biologically plausible and should remain so.
s = 1 # Number of initiations per minute
k = 0.02 # Degradation rate, 1/k = 50 min
k_off = 0 #Switching off rate, t_on = 1/k_off
k_on  = 1#/120 #Switching on rate, t_off = 1/k_on

p_on = 1 # Fraction of time the gene is ON

n_0 = s/k * p_on

t_end=10000 # Total simulation time in minutes, adjust if needed

parameters = [s, k, k_on, k_off]

In [ ]:
#Run Gillespie simulation
n, g, t = run_gillespie_telegraph(n_0, g_0, parameters, t_end, DT)

#Plot time series of mRNA and promoter state
plot_gillespie_telegraph(n, g, t)

#Plot distribution of mRNA counts
plot_distribution(n, title='mRNA distribution for Gene1')

#Compute stats
mean_n, expected_n, var_n, cv_n, p_on = compute_stats(n, g)

print(f"Mean level of expression: {mean_n:.2f}")
print(f"Expected number of molecules: {expected_n:.2f}")
print(f"Variance: {var_n:.2f}")
print(f"Coefficient of Variation (CV): {cv_n:.2f}")
print(f"Probability promoter is ON: {p_on:.2f}")

The Gene 1 case is the “cleanest” case and will be our reference case for this reason: when the promoter is always ON, transcription is continuous, so there are no transcriptional bursts. This results in very little noise and variability between cells is low (CV: 0.15).

#### Gene 2

We need p_on = 0.1, k_off = 1/15 has been chosen, then k_on needs to be 1/135.

In [ ]:
#initial conditions for the state (g,n) at time t=0
g_0 = 1 # Initial promoter state
DT = 0.1 # min
#if needed adjust DT so that it makes sense given the timescales of the system (depending on your rates)

#Parameters, these are biologically plausible and should remain so.
s = 1 # Number of initiations per minute
k = 0.02  # Degradation rate
k_off = 1/15 #Switching off rate, unit min^-1, t_on = 1/k_off 
k_on  = 1/135 #Switching on rate, unit min^-1, t_off = 1/k_on

t_on = 1/k_off
t_off = 1/k_on
p_on = t_on/(t_on+t_off) # Needs to be 0.1, k_off and k_on adjusted accordingly

n_0=s/k * p_on

t_end = 10000 # Total simulation time in minutes, adjust if needed

parameters = [s, k, k_on, k_off]

In [ ]:
#Run Gillespie simulation
n, g, t = run_gillespie_telegraph(n_0, g_0, parameters, t_end, DT)

#Plot time series of mRNA and promoter state
plot_gillespie_telegraph(n, g, t)

#Plot histogram of mRNA counts
plot_distribution(n, title='mRNA distribution for Gene2')

#Compute stats
mean_n, expected_n, var_n, cv_n, p_on = compute_stats(n, g)

print(f"Mean level of expression: {mean_n:.2f}")
print(f"Expected number of molecules: {expected_n:.2f}")
print(f"Variance: {var_n:.2f}")
print(f"Coefficient of Variation (CV): {cv_n:.2f}")
print(f"Probability promoter is ON: {p_on:.2f}")

The Gene 2 case will be the one displaying the classic transcriptional bursting noise. When the promoter is ON only 10% of the time, expression and transcription happens in bursts. This leads to a low mean epxression (4-5 molecules) and very large fluctuations (CV: 1.40, more or less).

#### Gene 2 Increased synthesis rate

To make Gene 2 ($P_{ON}=0.1$) achieve the same mean expression (more or less $\mu=50$) as Gene 1 ($P_{ON}=1.0$), where $\mu_1 = \frac{s}{k} = \frac{1}{0.02} = 50$, we can increase its synthesis rate ($s_2$) while keeping the degradation rate ($k_2=0.02$):$$\mu_2 = \frac{s_2}{k_2} \cdot P_{ON} = 50 \implies \frac{s_2}{0.02} \cdot 0.1 = 50 \implies 5 \cdot s_2 = 50 \implies s_2 = \mathbf{10}$$


In [ ]:
#initial conditions for the state (g,n) at time t=0
g_0 = 1 # Initial promoter state
DT = 0.1 # min
#if needed adjust DT so that it makes sense given the timescales of the system (depending on your rates)

#Parameters, these are biologically plausible and should remain so.
s = 10 # Number of initiations per minute
k = 0.02 # Degradation rate
k_off = 1/15 #Switching off rate, unit min^-1, t_on = 1/k_off
k_on  = 1/135 #Switching on rate, unit min^-1, t_off = 1/k_on

t_on = 1/k_off
t_off = 1/k_on
p_on = t_on/(t_on+t_off)

n_0=s/k * p_on

t_end = 10000 # Total simulation time in minutes, adjust if needed

parameters = [s, k, k_on, k_off]

In [ ]:
#Run Gillespie simulation
n, g, t = run_gillespie_telegraph(n_0, g_0, parameters, t_end, DT)

#Plot time series of mRNA and promoter state
plot_gillespie_telegraph(n, g, t)

#Plot distribution of mRNA counts
plot_distribution(n, title='mRNA distribution for Gene2 with increased synthesis rate')

#Compute stats
mean_n, expected_n, var_n, cv_n, p_on = compute_stats(n, g)

print(f"Mean level of expression: {mean_n:.2f}")
print(f"Expected number of molecules: {expected_n:.2f}")
print(f"Variance: {var_n:.2f}")
print(f"Coefficient of Variation (CV): {cv_n:.2f}")
print(f"Probability promoter is ON: {p_on:.2f}")

We have explained above that to match Gene 1’s mean expression while keeping the promoter mostly OFF, the synthesis rate of Gene 2 must increase 10 times. This means that the burst size also increases 10 times. Thus, this leads to huge bursts that extremely increase the variance between cells and still present a high Coefficient of Variation. We conclude that increasing the synthesis rate to match Gene 1's mean expression is not a suitable solution, as it is the worst scenario for variability.

#### Gene 2 Reduced degradation rate

To make Gene 2 ($P_{ON}=0.1$) achieve the same mean expression (more or less $\mu=50$) as Gene 1 ($P_{ON}=1.0$), where $\mu_1 = \frac{s}{k} = \frac{1}{0.02} = 50$, we can also decrease its degradation rate ($k_2$) while keeping the synthesis rate ($s=1$):$$\mu_2 = \frac{s}{k_2} \cdot P_{ON} = 50 \implies \frac{1}{k_2} \cdot 0.1 = 50 \implies 50 \cdot k_2 = 0.1 \implies k_2 = \frac{0.1}{50} = \mathbf{0.002}$$


In [ ]:
#initial conditions for the state (g,n) at time t=0
g_0 = 1 # Initial promoter state
DT = 0.1 # min
#if needed adjust DT so that it makes sense given the timescales of the system (depending on your rates)

#Parameters, these are biologically plausible and should remain so.
s = 1 # Number of initiations per minute
k = 0.002 # Degradation rate
k_off = 1/15 #Switching off rate unit min^-1, t_on = 1/k_off
k_on  = 1/135 #Switching on rate, unit min^-1, t_off = 1/k_on

t_on = 1/k_off
t_off = 1/k_on
p_on = t_on/(t_on+t_off)


n_0=s/k * p_on

t_end = 10000 # Total simulation time in minutes, adjust if needed

parameters = [s, k, k_on, k_off]

In [ ]:
#Run Gillespie simulation
n, g, t = run_gillespie_telegraph(n_0, g_0, parameters, t_end, DT)

#Plot time series of mRNA and promoter state
plot_gillespie_telegraph(n, g, t)

#Plot distribution of mRNA counts
plot_distribution(n, title='mRNA distribution for Gene2 with decreased degradation rate')

#Compute stats
mean_n, expected_n, var_n, cv_n, p_on = compute_stats(n, g)

print(f"Mean level of expression: {mean_n:.2f}")
print(f"Expected number of molecules: {expected_n:.2f}")
print(f"Variance: {var_n:.2f}")
print(f"Coefficient of Variation (CV): {cv_n:.2f}")
print(f"Probability promoter is ON: {p_on:.2f}")

We osbserve that, out of the two scenarios, decreasing Gene 2's degradation rate is the better alternative to match Gene 1's mean expression level, as the variance and noise are minimized. Although this strategy, like increasing the synthesis rate, creates a big burst size, it makes the mRNA molecules live longer. This longer lifetime smoothes the promoter's infrequent ON/OFF switching out, resulting in a lower variance and Coefficient of Variation, that are both closer to the Gene 1 reference values.

#### Question 2.1
1. Identify parameters that results in a Poisson distribution of mRNA numbers. Explain why these parameters produce a Poisson distribution. Show the empiral histogram and overlay an exact Poisson distribution. In this regime, explain how the parameters of the Telegraph model relate to the parameter of the Poisson distribution. *Hint*: $k_{off}=0$ with $g(t=0)=1$ is not the right answer. 

2. Find parameters where the mRNA distribution deviates from the Poisson. *Hint*: Look for a regime that yields a bimodal distribution. Explain your reasoning.  

####  Helper functions for the Poisson distribution

In [ ]:
# Poisson distribution
def Poisson(lam, counts):
    """
    Calculate the Poisson distribution probability for given counts.
    
    Parameters:
    lam: The average number of events (rate parameter).
    counts: Observed counts.

    Returns:
    np.array: Probabilities corresponding to the observed counts.

    Notes:
    - This function uses the log-form of the Poisson probability mass function.
    - The gammaln function is used for calculating the logarithm of the factorial term.

    """

    logp = counts * np.log(lam) - lam - gammaln(counts + 1)
    return np.exp(logp)

#### Answer 2.1.1

In [ ]:
#initial conditions for the state (g,n) at time t=0
g_0 = 1 # Initial promoter state

##Find Parameters giving Poisson distribution
s = 10 # Number of initiations per minute 
k = 0.1 # Degradation rate,  unit min^-1
k_off = 100 #Switching off rate, unit min^-1, t_on = 1/k_off
k_on  = 100 #Switching on rate, unit min^-1, t_off = 1/k_on

n_0 = 0 # Initial mRNA count

DT = 0.01 # min

t_end = 1000 * 1/k # Total simulation time in minutes, should be long, as in a Poisson process the k_on and k_off should be short compared the life time of mRNA. 

parameters = [s, k, k_on, k_off]
p_on = k_on / (k_on + k_off)

# Run the Gillepsie simulation
n, g, t = run_gillespie_telegraph(n_0, g_0, parameters, t_end, DT)

# Plot the theoretical Poisson distribution vs the simulated one
mean_n, expected_n, var_n, cv_n, p_on = compute_stats(n, g)

counts, bins = np.histogram(n, bins=range(0, max(n)+2), density=True)
bin_centers = 0.5 * (bins[:-1] + bins[1:])
poisson_probs = Poisson(expected_n, bin_centers)
plt.figure(figsize=(8,5))
plt.bar(bin_centers, counts, width=0.8, alpha=0.6, label='Simulated Distribution', color='lightblue', edgecolor='black')
plt.plot(bin_centers, poisson_probs, 'ro-', label='Theoretical Poisson', markersize=8)
plt.title('Simulated vs Theoretical Poisson Distribution')
plt.xlabel('Number of mRNA molecules')
plt.ylabel('Probability')
plt.legend()
plt.show()


#### How to chose the Poisson parameters ? 
The parameters that give a Poisson ditribution are those where there is a fast promoter switching regime. Therefore, we chose k_on and k_off larger than s and k. 

A rapid switch means that the promoter occupation is averaged as the pormoter is ON for a fraction pon = kon / (kon + koff). 

They should be short compared to the lifecycle of the mRNA. 

This is known as the Fast Switching Limit. When the promoter switches between 0 and 1 extremely rapidly (much faster than the time it takes for an mRNA molecule to degrade), the mRNA transcription machinery "sees" the promoter not as flickering, but as having a constant effective activity.

Instead of being fully ON (s) or fully OFF (0), the promoter behaves as if it is constantly producing at an average rate. The noise from the switching is averaged out, leaving only the intrinsic noise of birth and death, which results in a Poisson distribution.

##### **_Relationship to Poisson parameters_**
When the switch is fast the Telegraph model collapses into a one-step production/degradation model. Therefore, the parameter $\lambda$ is determined by the effective synthesis rate divided by the degradation rate. That is: 
$$ \lambda_{Poisson} = \frac{s_{eff}}{k} $$
where, 
$$ s_{eff} = s \cdot \frac{k_{on}}{k_{on} + k_{off}} $$




#### Answer 2.2.2


To deviate from the Poisson distribution, opposite conditions to those used for the Poisson regime should work. 
Therefore, we chose a high production rate, keep the same degradation rate but chose a very slow switching compared to the lifespan of the mRNA - therefore $k_{on}$ and $k_{off}$ are much smaller than $k$.


In [ ]:
#initial conditions for the state (g,n) at time t=0
g_0 = 1 # Initial promoter state

## Finding parameters giving a bimodal distribution
s = 50 # Number of initiations per minute 
k = 0.1 # Degradation rate,  unit min^-1
k_off = 0.01 #Switching off rate, unit min^-1, t_on = 1/k_off
k_on  = 0.01 #Switching on rate, unit min^-1, t_off = 1/k_on

n_0 = 0 # Initial mRNA count

DT = 0.01 # min

t_end = 100 * 1/k # Total simulation time in minutes, should be long, as in a Poisson process the k_on and k_off should be short compared the life time of mRNA. 

parameters = [s, k, k_on, k_off]
p_on = k_on / (k_on + k_off)

# Run the Gillepsie simulation
n, g, t = run_gillespie_telegraph(n_0, g_0, parameters, t_end, DT)

# Plot the theoretical Poisson distribution vs the simulated one
mean_n, expected_n, var_n, cv_n, p_on = compute_stats(n, g)

counts, bins = np.histogram(n, bins=range(0, max(n)+2), density=True)
bin_centers = 0.5 * (bins[:-1] + bins[1:])
poisson_probs = Poisson(expected_n, bin_centers)
plt.figure(figsize=(8,5))
plt.bar(bin_centers, counts, width=0.8, alpha=0.6, label='Simulated Distribution', color='lightblue', edgecolor='black')
plt.plot(bin_centers, poisson_probs, 'r-', label='Theoretical Poisson', markersize=8)
plt.title('Simulated vs Theoretical Poisson Distribution')
plt.xlabel('Number of mRNA molecules')
plt.ylabel('Probability')
plt.legend()
plt.show()


To distinguish betwen Poissonian and non-Poissonian gene expression, we can compute the Fano Factor. It is discribed as the variance divided by the mean $ \frac{\sigma^2}{\mu}$.
For the Telegraph model, the Fano Factor is defined as :
$$ F = 1 + \frac{s \cdot k_{off}}{(k_{on}+k_{off}) (k_{on}+k_{off}+k)} $$

If $F$ is close to 1 the distribution is close to the Poisson distribution, if $ F>1 $ then the distribution is overdispersed or bursty, and if $ F<1 $ the distribution is regular or oscillatory. 

In [ ]:
def theoretical_fano(s, k, k_on, k_off):
    """
    Calculates the analytical Fano Factor for the Telegraph model.
    """
    term_numerator = s * k_off
    term_denominator = (k_on + k_off) * (k_on + k_off + k)
    
    fano = 1 + (term_numerator / term_denominator)
    return fano

# Question 2.1
fano_1 = theoretical_fano(s=10, k=0.1, k_on=100, k_off=100)
print(f"Question 2.1 Fano Factor: {fano_1:.4f}")

# Question 2.2 
fano_2 = theoretical_fano(s=50, k=0.1, k_on=0.01, k_off=0.01)
print(f"Question 2.2 Fano Factor: {fano_2:.4f}")

In the first case the deviation is very small and the result is close to 1. Therefore the distribution is effectively Poisson. However for the second case, we can see that the parameters have been well chosen to differ from a Poisson distribution as the Fano Factor is larger than 1. We simulated a case of Bursty Expression. Here, the gene is not producing mRNA constantly it does by bursts. 

#### Question 3 (Promoter cycle)
Transcription in mammals often requires a more complicated telegraph model (promoter cycle), where the promoter state goes through an $ON$ state, and then sequentially goes through several $OFF_n$ state before returning to the ON state. For example with two $OFF$ states, it would look like: $ON\,(g=1)\longrightarrow OFF_1 (g=2) \longrightarrow OFF_2 (g=3) \longrightarrow ON (g=1)$. This is related to the fact that transcription events often show a refractory period, *i.e.* promoter tend to say off for some amount of time before turning back on. 

1. Modify the above code to implement the promoter cycle with two $OFF$ state. First, use rates as in the example above with the following modifications: $s = 20$, $k = 0.02$, $k_{on-off_1} = 1/15$, $k_{off_1-off_2} = 1/60$ and $k_{off_2-on}  = 1/60$. Show some representative simulations. Plot both the mRNA and promoter state and comment.

2.  Study the distribution of the durations of each $OFF$ interval separately and study the distribution of the total duration of the two $OFF$ intervals. Focus on the shape of the distributions. Explain your findings.

3. (Bonus). Can you explain mathematically and biologically the shape of the distribution ? What would happen if you include more $OFF_n$ states?

#### Question 3.1

In [ ]:
# States: 1 = ON, 2 = OFF1, 3 = OFF2
def run_gillespie_promoter_cycle(n0, g0, parameters, t_end, DT):
    """
    Simulate promoter cycle 1 -> 2 -> 3 -> 1 with mRNA production in state 1.
    parameters: (s, k, k_1to2, k_2to3, k_3to1)
    Returns lists (n_print, g_print, t_print) sampled every DT,
    AND a dictionary with exact durations of OFF states.
    """
    n = int(n0)
    g = int(g0)
    t = 0.0
    tp = 0.0

    n_print = []
    g_print = []
    t_print = []
    
    # To record exact durations
    durations = {'OFF1': [], 'OFF2': []}
    t_last_switch = 0.0

    s, k, k_1to2, k_2to3, k_3to1 = parameters

    while t < t_end:
        rates = []
        reactions = []

        # Define possible reactions depending on promoter state
        if g == 1:
            rates.extend([k_1to2, s, k * n])   # 1->2, synthesis, degradation
            reactions.extend(['1_to_2', 'synth', 'deg'])
        elif g == 2:
            rates.extend([k_2to3, k * n])      # 2->3, degradation
            reactions.extend(['2_to_3', 'deg'])
        elif g == 3:
            rates.extend([k_3to1, k * n])      # 3->1, degradation
            reactions.extend(['3_to_1', 'deg'])

        rates = np.array(rates, dtype=float)
        rates_tot = rates.sum()
        if rates_tot <= 0:
            break

        # Sample time to next reaction
        dt = np.random.exponential(scale=1.0 / rates_tot)
        t += dt

        # Record outputs at regular DT intervals (same behaviour as previous cells)
        while tp < t and tp <= t_end:
            n_print.append(n)
            g_print.append(g)
            t_print.append(tp)
            tp += DT

        # Choose reaction
        r = np.random.random()
        cum = np.cumsum(rates) / rates_tot
        idx = np.searchsorted(cum, r)
        chosen = reactions[int(idx)]

        # Execute the chosen reaction
        if chosen == 'synth':
            n += 1
        elif chosen == 'deg':
            if n > 0:
                n -= 1
        elif chosen == '1_to_2':
            # Transition 1 -> 2
            # We are leaving state 1. We don't need to record duration of state 1 for this exercise, 
            # but we mark the start of state 2.
            t_last_switch = t
            g = 2
        elif chosen == '2_to_3':
            # Transition 2 -> 3
            # We are leaving state 2 (OFF1). Record duration.
            durations['OFF1'].append(t - t_last_switch)
            t_last_switch = t
            g = 3
        elif chosen == '3_to_1':
            # Transition 3 -> 1
            # We are leaving state 3 (OFF2). Record duration.
            durations['OFF2'].append(t - t_last_switch)
            t_last_switch = t
            g = 1

    return n_print, g_print, t_print, durations

In [ ]:
def plot_promoter_cycle(n, g, t):
    """
    Plot mRNA and discrete promoter states (1,2,3).
    """
    fig, axs = plt.subplots(2, figsize=(14,5), sharex=True)

    axs[0].plot(t, n, color='darkblue')
    axs[0].set_ylabel('# mRNA molecules')
    axs[0].set_title('mRNA and promoter state vs time')
    
    # Plotting the promoter activity over time
    # Use steps-post to clearly show discrete states
    axs[1].step(t, g, where='post', color='teal')
    axs[1].set_ylabel('State (1=ON, 2=OFF1, 3=OFF2)')
    axs[1].set_xlabel('Time (min)')
    axs[1].set_yticks([1, 2, 3])
    axs[1].set_yticklabels(['ON', 'OFF1', 'OFF2'])
    axs[1].grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Gillespie simulator for a promoter cycle with two OFF states:

# Simulations with the requested rates
# Parameters given above
s = 20
k = 0.02
k_1to2 = 1/15.0    # ON -> OFF1
k_2to3 = 1/60.0    # OFF1 -> OFF2
k_3to1 = 1/60.0    # OFF2 -> ON

# Initial conditions and simulation settings
params = (s, k, k_1to2, k_2to3, k_3to1)
n0 = 0
g0 = 1
DT = 0.1
t_end = 10000 

# Run simulation
n_traj, g_traj, t_traj, durations = run_gillespie_promoter_cycle(n0, g0, params, t_end, DT)

# Compute simple stats and print them
n_arr = np.array(n_traj)
p_on = np.mean([1 if gg==1 else 0 for gg in g_traj])
mean_n = n_arr.mean() if n_arr.size>0 else 0.0
var_n = n_arr.var() if n_arr.size>0 else 0.0
cv_n = np.sqrt(var_n)/mean_n if mean_n>0 else np.nan

print(f'Simulation parameters: s={s}, k={k}, k_1to2={k_1to2:.4f}, k_2to3={k_2to3:.4f}, k_3to1={k_3to1:.4f}')
print(f'Total simulated time: {t_end} min, DT={DT} min')
print(f'Mean mRNA: {mean_n:.2f}, Variance: {var_n:.2f}, CV: {cv_n:.2f}, P(ON): {p_on:.3f}')

The plot shows bursts of mRNA when the promoter hits state ON; OFF1 and OFF2 durations shape burst spacing. The ON events are separated by longer OFF intervals (set by 1/60 min ~ 60 min average for the two transitions), and mRNA counts are rising during ON and decaying in OFF states.

##### Other simulations

In [ ]:
# Other scenarios to explore
scenarios = [
    {
        "name": "Symmetric (Perfect Timer)\nEqual steps create a clear peak",
        "params": [20, 0.02, 1/15, 1/60, 1/60] 
    },
    {
        "name": "Asymmetric (Hidden Step)\nOne slow step dominates -> Exponential-like",
        "params": [20, 0.02, 1/15, 1/100, 1.0] # OFF1->OFF2 is slow, OFF2->ON is instant
    },
    {
        "name": "Fast Cycle (Noisy)\nFast steps -> Short Refractory Period",
        "params": [20, 0.02, 1/5, 1/10, 1/10] # Fast switching
    }
]

In [ ]:
# Run all scenarios and plot in one figure with 6 subplots (6 rows x 1 column)
fig, axs = plt.subplots(6, 1, figsize=(12, 18), sharex=True)

for i, scenario in enumerate(scenarios):
    s, k, k_1to2, k_2to3, k_3to1 = scenario["params"]
    
    # Run simulation
    n_traj, g_traj, t_traj, durations = run_gillespie_promoter_cycle(n0, g0, scenario["params"], t_end, DT)
    
    # Calculate Stats
    n_arr = np.array(n_traj)
    p_on = np.mean([1 if gg==1 else 0 for gg in g_traj])
    mean_n = n_arr.mean() if n_arr.size>0 else 0.0
    var_n = n_arr.var() if n_arr.size>0 else 0.0
    cv_n = np.sqrt(var_n)/mean_n if mean_n>0 else np.nan        

    # Plot mRNA trace (Top subplot for this scenario)
    # Scenario 1: rows 0,1. Scenario 2: rows 2,3. Scenario 3: rows 4,5.
    ax_mrna = axs[2*i]
    ax_mrna.plot(t_traj, n_traj, color='darkblue', label='mRNA')
    ax_mrna.set_ylabel('# mRNA molecules')
    ax_mrna.set_title(f"Scenario: {scenario['name']} (mRNA)")
    ax_mrna.grid(True, alpha=0.3)
    
    # Plot Promoter State (Bottom subplot for this scenario)
    ax_prom = axs[2*i+1]
    ax_prom.step(t_traj, g_traj, where='post', color='teal', lw=1, label='Promoter State')
    ax_prom.set_yticks([1, 2, 3])
    ax_prom.set_yticklabels(['ON', 'OFF1', 'OFF2'])
    ax_prom.set_ylabel('Promoter State')
    ax_prom.set_ylim(0.5, 3.5)
    ax_prom.invert_yaxis()
    ax_prom.set_title(f"Scenario: {scenario['name']} (Promoter)")
    ax_prom.grid(True, alpha=0.3)
    
    # Add statistics text box to the right of the promoter plot
    stats_text = (
        f"Mean: {mean_n:.2f}\n"
        f"Var: {var_n:.2f}\n"
        f"CV: {cv_n:.2f}\n"
        f"P(ON): {p_on:.2f}"
    )
    # Place text box in upper right of the promoter plot
    ax_prom.text(1.02, 0.5, stats_text, transform=ax_prom.transAxes, 
                verticalalignment='center', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

axs[-1].set_xlabel('Time [min]')
plt.suptitle('Promoter Cycle Simulations with Different Kinetics', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

It's possible to see that as the switching becomes faster (ie the OFF states become shorter) the probabilty of being ON increases (from 0.11 to 0.21) which leads to a higher mean mRNA count, which was expected. 

When looking at the variance it's possible to see that the fast cycle is the most stable, with the lowest relative noise (CV) while the other two cycles are more heterogeneous and bursty. 

#### Question 3.2

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import expon, gamma

In [ ]:
def analyze_off_durations(durations):
    """
    Analyzes the duration of OFF states (g=2 and g=3) and the total OFF time.
    Takes the dictionary of exact durations returned by the Gillespie simulation.
    """
    durations_off1 = np.array(durations['OFF1'])
    durations_off2 = np.array(durations['OFF2'])
    
    # For Total OFF duration, we need to sum consecutive OFF1 and OFF2
    # We truncate to the minimum length in case the sim ended in the middle of a cycle
    min_len = min(len(durations_off1), len(durations_off2))
    durations_total_off = durations_off1[:min_len] + durations_off2[:min_len]

    # --- Plotting ---
    plt.figure(figsize=(15, 5))

    # 1. Distribution of OFF_1
    plt.subplot(1, 3, 1)
    plt.hist(durations_off1, bins=30, density=True, alpha=0.6, color='skyblue', label='Simulated')
    # Overlay theoretical Exponential
    if len(durations_off1) > 0:
        mean_off1 = np.mean(durations_off1)
        x_range = np.linspace(0, max(durations_off1), 100)
        plt.plot(x_range, expon.pdf(x_range, scale=mean_off1), 'r--', lw=2, label='Exp. Fit')
        plt.title(f'Duration of $OFF_1$\n(Mean: {mean_off1:.1f} min)')
    else:
        plt.title('Duration of $OFF_1$\n(No events)')
    plt.xlabel('Time (min)')
    plt.legend()

    # 2. Distribution of OFF_2
    plt.subplot(1, 3, 2)
    plt.hist(durations_off2, bins=30, density=True, alpha=0.6, color='salmon', label='Simulated')
    # Overlay theoretical Exponential
    if len(durations_off2) > 0:
        mean_off2 = np.mean(durations_off2)
        x_range = np.linspace(0, max(durations_off2), 100)
        plt.plot(x_range, expon.pdf(x_range, scale=mean_off2), 'k--', lw=2, label='Exp. Fit')
        plt.title(f'Duration of $OFF_2$\n(Mean: {mean_off2:.1f} min)')
    else:
        plt.title('Duration of $OFF_2$\n(No events)')
    plt.xlabel('Time (min)')
    plt.legend()

    # 3. Distribution of Total OFF (OFF1 + OFF2)
    plt.subplot(1, 3, 3)
    plt.hist(durations_total_off, bins=30, density=True, alpha=0.6, color='purple', label='Simulated')
    
    # Overlay theoretical Gamma (Erlang) distribution
    if len(durations_total_off) > 0:
        x_range = np.linspace(0, max(durations_total_off), 100)

        # We fit the gamma to the data to see the shape match
        # Because in the Gillespie simulation, the waiting times for a single step are exponentially distributed,
        # Therefore, the total OFF durations should follow a Gamma (Erlang) distribution.
        params = gamma.fit(durations_total_off, floc=0) 
        plt.plot(x_range, gamma.pdf(x_range, *params), 'y--', lw=2, label='Gamma Fit')
        plt.title(f'Total OFF Duration ($OFF_1 + OFF_2$)\n(Mean: {np.mean(durations_total_off):.1f} min)')
    
    else:
        plt.title('Total OFF Duration\n(No events)')
        
    plt.suptitle('Analysis of OFF State Durations', fontsize=16)
    plt.xlabel('Time (min)')
    plt.legend()
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

In [ ]:
# Run with the simulation data with the parameters from the first scenario
"""
The parameter values used in the simulation:
s = 20
k = 0.02
k_1to2 = 1/15.0    # ON -> OFF1
k_2to3 = 1/60.0    # OFF1 -> OFF2
k_3to1 = 1/60.0    # OFF2 -> ON 
"""
analyze_off_durations(durations)

In [ ]:
# Doing the same for the 3 other scenarios
for i, scenario in enumerate(scenarios):
    s, k, k_1to2, k_2to3, k_3to1 = scenario["params"]
    
    # Run simulation
    n_traj, g_traj, t_traj, durations = run_gillespie_promoter_cycle(n0, g0, scenario["params"], t_end, DT)
    
    print(f"Analyzing OFF durations for scenario {i+1}: {scenario['name']}")
    analyze_off_durations(durations)

In [ ]:
# Superimpose the total OFF durations from all scenarios
plt.figure(figsize=(10, 6))
for i, scenario in enumerate(scenarios):
    s, k, k_1to2, k_2to3, k_3to1 = scenario["params"]
    
    # Run simulation
    n_traj, g_traj, t_traj, durations = run_gillespie_promoter_cycle(n0, g0, scenario["params"], t_end, DT)
    
    # Calculate total OFF durations
    durations_off1 = np.array(durations['OFF1'])
    durations_off2 = np.array(durations['OFF2'])
    min_len = min(len(durations_off1), len(durations_off2))
    durations_total_off = durations_off1[:min_len] + durations_off2[:min_len]

    # Overlay theoretical Gamma (Erlang) distribution
    x_range = np.linspace(0, max(durations_total_off), 100)
    
    # We fit the gamma to the data to see the shape match
    # Because in the Gillespie simulation, the waiting times for a single step are exponentially distributed,
    # Therefore, the total OFF durations should follow a Gamma (Erlang) distribution.
    params = gamma.fit(durations_total_off, floc=0) 
    plt.plot(x_range, gamma.pdf(x_range, *params), '--', lw=2, label=f'Scenario {i+1}: {scenario["name"]}')
    plt.legend(f'Total OFF Duration ($OFF_1 + OFF_2$)\n(Mean: {np.mean(durations_total_off):.1f} min)')
        
plt.xlabel('Total OFF Duration (min)')
plt.ylabel('Density')
plt.title('Superimposed Total OFF Duration Distributions')
plt.legend()
plt.show()

The Scenario 1 curve has a Gamma-shaped distribution, which proves that the existence of a Refractory Period. This means that once the system turns OFF, the probability that it turns back ON **immediately is 0.** 

The Scenario 2 can yield exponential looking distributions _(either starting at 0 and very quickly going up to a maximum and then decaying, or starting directly from that maximum)_ hinting that when the steps are widely separated the system loses its memory and it collapses into a 2-step Telegraph model. 

The Scenario 3, a fast switching scenario, has a very short total OFF time compared to the others. This is because with fast switches the gene will turn back ON before the mRNA from the previous burst finishes degrading. 

This leads us to believe that if a cell needed a precise delay between bursts (like for circadian rhythms), it should have multiple sequential steps with similar rates rather than a slow step. 

#### Question 3.3

The sum of n exponential steps gives a Gamma distribution. This creates a Refractory Period or "molecular memory", which means that once the gene turns off, it cannot turn back on immediately. It is "forced" to travel through the cycle (OFF 
1
​	
 →OFF 
2
​	
 ).

If we added more OFF states (OFF 
1
​	
 →⋯→OFF 
10
​	
 ), the Gamma distribution would become sharper and more symmetric, eventually approaching a Gaussian distribution. This would turn a random, noisy switch into a precise biological timer or oscillator.